In [1]:
import sys, os, time
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib tk
import matplotlib.pyplot as plt

import tnt_util as util
from tnt_util import adict, idict, xset, collate, load, render_dict, save, Logger, seq_iterate
from tnt_setup import init_gamestate, setup_phase
from tnt_cards import load_card_decks, draw_cards
from collections import namedtuple
import random
from itertools import chain, product
from tnt_units import load_unit_rules
import tnt_setup as setup

from flask_app import *

In [2]:
ping()

'Backend active: use "init" to init game'

In [3]:
player = 'Axis'

In [4]:
out = format_msg_to_python(init_game(debug=False, player=player))

Beginning phase: Setup


In [5]:
path = 'saves/setup_complete.json'
load(path)

Game loaded


In [6]:
player = 'West'

In [7]:
player = 'USSR'

In [8]:
player

'USSR'

In [17]:
out = format_msg_to_python(get_status(player))

In [18]:
if 'log' in out:
    print(out.log)

USSR spends 1 production on building a new cadre in Stalingrad (1 production remaining)
USSR spends 1 production on building a new cadre in Kiev (0 production remaining)
USSR draws 0 action cards (now holding 6 cards)
USSR draws 0 investment cards (now holding 6 cards)
USSR is done with production



In [19]:
render_dict(out)

In [5]:
# actions = list(util.decode_actions(out.actions))
# actions

In [6]:
# action = random.choice(actions)
# action

In [7]:
#out = take_action(player, action)

In [8]:
#out = format_msg_to_python(take_action(player, ('Britain', 'Paris', 'Infantry')))

In [7]:
for player in ['Axis', 'West', 'USSR']:
    out = format_msg_to_python(get_status(player))
    for _ in range(25):
        if 'error' in out:
            print(out.error)
            break
        elif 'actions' not in out:
            #render_dict(out)
            print('Player {} is waiting'.format(player))
            break
        action = random.choice(list(util.decode_actions(out.actions)))
        print('--- Playing',action)
        out = format_msg_to_python(take_action(player, action))
        if 'log' in out and len(out.log)>0:
            print(out.log)

--- Playing ('Italy', 'Sicily', 'AirForce')
Axis spends 1 production on building a new cadre in Sicily (10 production remaining)

--- Playing ('Italy', 'Sardinia', 'Submarine')
Axis spends 1 production on building a new cadre in Sardinia (9 production remaining)

--- Playing ('Italy', 'Rome', 'Tank')
Axis spends 1 production on building a new cadre in Rome (8 production remaining)

--- Playing (127,)
Axis spends 1 production on upgrading a cadre in Munich (7 production remaining)

--- Playing (117,)
Axis spends 1 production on upgrading a cadre in Ruhr (6 production remaining)

--- Playing ('Italy', 'Sardinia', 'Submarine')
Axis spends 1 production on building a new cadre in Sardinia (5 production remaining)

--- Playing (169,)
Axis spends 1 production on upgrading a cadre in Sardinia (4 production remaining)

--- Playing ('Germany', 'Konigsberg', 'Fleet')
Axis spends 1 production on building a new cadre in Konigsberg (3 production remaining)

--- Playing ('Italy', 'Rome', 'Fleet')
Axi

In [7]:
G = get_G()
render_dict(G)

In [1]:
W = get_waiting_actions()
render_dict(W)

NameError: name 'get_waiting_actions' is not defined

In [11]:
for o in seq_iterate(G.players, [None, 'homeland', None], end=True):
    print(o)

('Axis', 'Germany', {'Konigsberg', 'Ruhr', 'Berlin', 'Munich'})
('Axis', 'Italy', {'Taranto', 'Sicily', 'Venice', 'Sardinia', 'Milan', 'Rome'})
('USSR', 'USSR', {'Stalingrad', 'Kazakhstan', 'Gorky', 'Belorussia', 'Voronezh', 'Archangel', 'Kuban', 'Odessa', 'Kharkov', 'Bryansk', 'Murmansk', 'Penza', 'Georgia', 'Moscow', 'Perm', 'Leningrad', 'Western_Siberia', 'Baku', 'Urals', 'Sevastopol', 'Kiev', 'Grozny', 'Turkmenistan', 'Vologda', 'Ufa'})
('West', 'Britain', {'London', 'Glasgow'})
('West', 'France', {'Paris', 'Lorraine', 'Marseille', 'Gascony'})


In [7]:
# path = save('setup_complete.json')
# path

Game saved


'saves/setup_complete.json'

In [7]:
path = 'saves/init_complete.json'
load(path)

Game loaded


In [2]:
a = adict()
load_unit_rules(a)
render_dict(a)